__Demultiplexing the two sequencing libraries using barcode splitter and then trimmatic(?). Following Michelle's demultiplexing steps on https://github.com/pinskylab/genomics/blob/df5c254a0e097da18f49532f748a5a2ae5e1299f/Protocols/01_hiseq_workflow.md as a rough guide, and barcode_splitter.py documentation: https://pypi.org/project/barcode-splitter/__

__Start with sequencing library 1__

In [9]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/bcsplit/


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/noveseq_2021_03_26_SEQ1/bcsplit


__Each capture (1-16) was a PCR pool, first separate out by PCR pool__

In [10]:
#run within screen annotate
!/local/home/katrinac/ClownfishGWAS/bioinformatics_scripts/paired_sequence_utils/barcode_splitter.py --bcfile /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/PCRCapturePoolsSP1.tsv /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_2_Index_Read_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_3_Index_Read_passed_filter.fastq.gz --idxread 2 --suffix .fastq  

OSError: Background processes not supported.

In [ ]:
#run within screen annotate
!/local/home/katrinac/ClownfishGWAS/bioinformatics_scripts/paired_sequence_utils/barcode_splitter.py --bcfile /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/PCRCapturePoolsSP2.tsv /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_1_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_2_Index_Read_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_3_Index_Read_passed_filter.fastq.gz --idxread 2 --suffix .fastq  